In [ ]:
import utils
import cleaning
import pandas as pd
import ast
import re


In [ ]:
cleaning.recipes_ratings_merged_cleaned()
df = pd.read_csv("../data/Processed/recipes with ratings.csv")

In [ ]:
df.sample(5)

In [ ]:
# Check the first element of each column to determine if they are strings or lists
first_tags = df['tags'].iloc[0]
first_ingredients = df['ingredients'].iloc[0]
first_nutrition = df['nutrition'].iloc[0]
first_steps = df['steps'].iloc[0]

# Print the type of the first element of each column
types = {
    'tags': type(first_tags),
    'ingredients': type(first_ingredients),
    'nutrition': type(first_nutrition),
    'steps': type(first_steps)
}

# Display the types
for column, typ in types.items():
    print(f'The first element of the {column} column is of type: {typ}')

# Convert the 'tags' column from string to list
if isinstance(df['tags'].iloc[0], str):
    df['tags'] = df['tags'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'ingredients' column from string to list
if isinstance(df['ingredients'].iloc[0], str):
    df['ingredients'] = df['ingredients'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'nutrition' column from string to list
if isinstance(df['nutrition'].iloc[0], str):
    df['nutrition'] = df['nutrition'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

# Convert the 'steps' column from string to list
if isinstance(df['steps'].iloc[0], str):
    df['steps'] = df['steps'].apply(lambda x: x.strip('[]').replace("'", '').split(', '))

In [ ]:
# Function to categorize time into intervals
def categorize_time(minutes):
    if minutes > 90:
        return 'More than 1h 30min'
    else:
        interval = (minutes // 15) * 15
        return f'About {interval} min' if interval != 0 else 'About 15 min'

# Apply the function to create a new column
df['cookking_time'] = df['minutes'].apply(categorize_time)
df


In [ ]:
nutrition_df = cleaning.create_nutrition_df(df, 'nutrition', 'recipe_id')


In [ ]:
# Function to categorize number of ingredients into intervals
def categorize_ingredients(n_ingredients):
    if n_ingredients <= 5:
        return '0-5'
    elif 5 < n_ingredients <= 10:
        return '5-10'
    elif 10 < n_ingredients <= 20:
        return '10-20'
    elif 20 < n_ingredients <= 30:
        return '20-30'
    elif 30 < n_ingredients <= 40:
        return '30-40'
    else:
        return '> 40'

# Apply the function to create a new column
df['ingredient_category'] = df['n_ingredients'].apply(categorize_ingredients)

In [ ]:
def aggregate_unique_lists(df, column_name):
    """
    Aggregates all the lists from a specified column into a single list of unique elements.

    Parameters:
    df (pd.DataFrame): DataFrame to process.
    column_name (str): Name of the column containing lists or string representations of lists.

    Returns:
    list: A single list containing all unique elements from the lists in the specified column.
    """

    unique_elements = set()
    for item in df[column_name].dropna():
        # Convert string representation of list to actual list if necessary
        if isinstance(item, str):
            try:
                item = ast.literal_eval(item)
            except ValueError:
                continue  # Skip items that cannot be converted

        # Check if the item is a list and add its elements to the set
        if isinstance(item, list):
            unique_elements.update(item)
        else:
            unique_elements.add(item)

    return list(unique_elements)



In [ ]:

def find_dairy_ingredients(ingredients_list):
    # Define the dairy categories with their keywords and exclusion terms
    dairy_categories = {
    'milk': {
        'keywords': [r'\bmilk\b'],
        'exclusions': ['almond', 'coconut', 'non-dairy', 'oat', 'rice', 'soy']
    },
    'yogurt': {
        'keywords': [r'\byogurt\b', r'\byoghurt\b'],
        'exclusions': []
    },
    'butter': {
        'keywords': [r'\bbutter\b'],
        'exclusions': ['almond', 'apple', 'butterfly', 'butter substitute', 'butterball', 
                       'buttercup', 'butterfish', 'butterflied', 'butternut', "i can't believe it's not butter",
                       'butter beans', 'lettuce', 'peas', 'coconut', 'vegan']
    },
    'cream': {
        'keywords': [r'\bcream\b'],
        'exclusions': ['almond', 'coconut', 'non-dairy', 'non-creamy', 'soy', 'vegan']
    },
    'cheese': {
        'keywords': [r'\bcheese\b'],
        'exclusions': ['non-dairy', 'soy', 'vegan', 'vegetarian']
    },
    'ice_cream': {
        'keywords': [r'\bice cream\b', r'\bgelato\b'],
        'exclusions': ['non-dairy', 'almond', 'coconut', 'soy']
    },
    'curd': {
        'keywords': [r'\bcurd\b'],
        'exclusions': ['lemon', 'lime']
    },
    'whey': {
        'keywords': [r'\bwhey\b'],
        'exclusions': []
    },
    'casein': {
        'keywords': [r'\bcasein\b'],
        'exclusions': []
    },
    'ghee': {
        'keywords': [r'\bghee\b'],
        'exclusions': []
    },
    'kefir': {
        'keywords': [r'\bkefir\b'],
        'exclusions': []
    },
    'paneer': {
        'keywords': [r'\bpaneer\b'],
        'exclusions': []
    },
    'condensed_milk': {
        'keywords': [r'\bcondensed milk\b'],
        'exclusions': ['non-dairy', 'soy']
    },
    'evaporated_milk': {
        'keywords': [r'\bevaporated milk\b'],
        'exclusions': ['non-dairy', 'soy']
    },
    'milk_powder': {
        'keywords': [r'\bmilk powder\b', r'\bdried milk\b', r'\bpowdered milk\b'],
        'exclusions': ['non-dairy', 'soy']
    }
    # You can continue to add more categories if needed
}


    # Function to compile regex and find matches
    def find_matches(category, ingredients):
        keywords_regex = re.compile('|'.join(category['keywords']), re.IGNORECASE)
        exclusions_regex = re.compile('|'.join(category['exclusions']), re.IGNORECASE) if category['exclusions'] else None
        return [i for i in ingredients if keywords_regex.search(i) and (not exclusions_regex or not exclusions_regex.search(i))]

    # Aggregate all dairy ingredients
    dairy = []
    for category in dairy_categories.values():
        dairy.extend(find_matches(category, ingredients_list))

    # Remove duplicates and sort
    dairy = list(set(dairy))
    dairy.sort()

    return dairy

# Example usage
dairy_ingredients = find_dairy_ingredients(ingredients_list)


In [ ]:
# Calculate the median number of ratings across all recipes
# This value serves as a threshold to differentiate between recipes with many vs few ratings
m = recipes_with_ratings['number_of_ratings'].quantile(0.50)

# Compute the mean average rating across all recipes
# This represents the typical rating a recipe receives in the dataset
C = recipes_with_ratings['average_rating'].mean()

# Define a function to calculate the weighted rating for each recipe
def weighted_rating(x, m=m, C=C):
    v = x['number_of_ratings']  # Number of ratings for the recipe
    R = x['average_rating']     # Average rating of the recipe
    # Weighted rating formula: balances the recipe's average rating (R) with the mean rating (C)
    return (v/(v+m) * R) + (m/(m+v) * C)

# Apply the weighted rating formula to each recipe in the filtered DataFrame
recipes_with_ratings['weighted_rating'] = recipes_with_ratings.apply(weighted_rating, axis=1)